In [1]:
from geosyspy import Geosys
import os
from dotenv import load_dotenv
import datetime as dt
from dateutil.relativedelta import relativedelta
import logging
from geosyspy.utils.constants import *


logger = logging.getLogger()
logger.setLevel(logging.INFO)

# read .env file
load_dotenv()

API_CLIENT_ID = os.getenv('API_CLIENT_ID')
API_CLIENT_SECRET = os.getenv('API_CLIENT_SECRET')
API_USERNAME = os.getenv('API_USERNAME')
API_PASSWORD = os.getenv('API_PASSWORD')

client = Geosys(API_CLIENT_ID, API_CLIENT_SECRET, API_USERNAME, API_PASSWORD, Env.PROD, Region.NA)

polygon = "POLYGON((-91.17523978603823 40.29787117039518,-91.17577285022956 40.29199489606421,-91.167613719932 40.29199489606421,-91.1673028670095 40.29867040193312,-91.17523978603823 40.29787117039518))"
today = dt.date.today()
year_ago = dt.date.today() + relativedelta(months=-12)

INFO:root:Authenticated


In [ ]:
# Get aggregated NDVI time series
client.get_time_series(polygon, year_ago, today, collection=SatelliteImageryCollection.MODIS, indicators=["NDVI"])

In [ ]:
# Get aggregated EVI time series
client.get_time_series(polygon, year_ago, today, collection=SatelliteImageryCollection.MODIS, indicators=["EVI"])

In [ ]:
# Get aggregated 'Forecast daily' time series
indicators = ['Precipitation', 'Temperature','Date']
point = "POINT (0.0872845021171696 43.69457564315705)"
client.get_time_series(point, dt.date.today(), dt.date.today() + relativedelta(days=+5), collection=WeatherTypeCollection.WEATHER_FORECAST_DAILY, indicators=indicators)

In [ ]:
# Get aggregated 'Forecast hourly' time series
indicators = ['Precipitation', 'Temperature']
point = "POINT (0.0872845021171696 43.69457564315705)"
client.get_time_series(point, dt.date.today(), dt.date.today() + relativedelta(days=+2), collection=WeatherTypeCollection.WEATHER_FORECAST_HOURLY, indicators=indicators)

In [ ]:
# Get aggregated 'Historical daily' time series
indicators = ['Precipitation', 'Temperature']
start_date = dt.datetime.strptime("2022-01-01", "%Y-%m-%d")
end_date = dt.datetime.strptime("2022-02-01", "%Y-%m-%d")
client.get_time_series(polygon, start_date, end_date, collection=WeatherTypeCollection.WEATHER_HISTORICAL_DAILY, indicators=indicators)

In [ ]:
# Get satellite image time series for Modis NDVI
client.get_satellite_image_time_series(polygon, year_ago, today, collections=[SatelliteImageryCollection.MODIS], indicators=["NDVI"])

In [ ]:
# Get satellite image time series for LANDSAT_8 and SENTINEL_2 Reflectance
time_series_xarr = client.get_satellite_image_time_series(polygon, year_ago, today, collections=[SatelliteImageryCollection.SENTINEL_2, SatelliteImageryCollection.LANDSAT_8], indicators=["Reflectance"])
time_series_xarr


In [ ]:
# Get satellite image time series for LANDSAT_8 and SENTINEL_2 NDVI 
# list of available indicators: Reflectance, NDVI, EVI, GNDVI, NDWI, CVI, CVIn, LAI
ndvi_time_series_xarr = client.get_satellite_image_time_series(polygon, year_ago, today, collections=[SatelliteImageryCollection.SENTINEL_2, SatelliteImageryCollection.LANDSAT_8], indicators=["ndvi"])
ndvi_time_series_xarr

In [ ]:
# Display cumulative NDVI from last result

import numpy as np
import matplotlib.pyplot as plt

polygon_ndvi_ds = ndvi_time_series_xarr['ndvi'].sortby('time')
time_coords = polygon_ndvi_ds['time']

# exclude Nan values to caluculate mean
masked_dataarray = polygon_ndvi_ds.where(~np.isnan(polygon_ndvi_ds))

# NDVI mean calculattion
mean_ndvi = masked_dataarray.mean(dim=['x', 'y'])

# Cumulative NDVI calculation 
polygon_cumul_ndvi = mean_ndvi.cumsum(dim='time')
polygon_cumul_ndvi 

# build & display cumlative Ndvi graph
plt.plot(time_coords, polygon_cumul_ndvi)
plt.xlabel('Time')
plt.ylabel('NDVI Cumul')
plt.title('NVDI Cumul by date')
plt.show()

In [ ]:
import matplotlib
time_series_xarr.reflectance.clip(0,1).sel(band='Green').plot(x="x", y="y", col="time", col_wrap=3, figsize=(20,20))

In [ ]:
# Get coverage for the polygon
coverage_info_df, images_references = client.get_satellite_coverage_image_references(polygon, year_ago, today, collections=[SatelliteImageryCollection.SENTINEL_2, SatelliteImageryCollection.LANDSAT_8, SatelliteImageryCollection.LANDSAT_9])
coverage_info_df

In [ ]:
images_references

In [ ]:
# Download and save a specific image
client.download_image(images_references[('2023-05-03', 'SENTINEL_2')])


In [ ]:
# Define a data schema in Analytics Fabrics
schema = {
    "NDVI": "double"
}
schema_id = "GeosysPy_NDVI"
client.create_schema_id(schema_id, schema)

In [ ]:
# Get metrics in Analytics Fabric

start_date = dt.datetime.strptime("2022-01-24", "%Y-%m-%d")
end_date = dt.datetime.strptime("2022-03-01", "%Y-%m-%d")
schema_id = "LAI_RADAR"
polygon = "POLYGON((-52.72591542 -18.7395779,-52.72604885 -18.73951122,-52.72603114 -18.73908689,-52.71556835 -18.72490316,-52.71391916 -18.72612966,-52.71362802 -18.72623726,-52.71086473 -18.72804231,-52.72083542 -18.74173696,-52.72118937 -18.74159174,-52.72139229 -18.7418552,-52.72600257 -18.73969719,-52.72591542 -18.7395779))"
client.get_metrics(polygon, schema_id, start_date, end_date)

In [ ]:
# Get time serie
start_date = dt.datetime.strptime("2018-12-30", "%Y-%m-%d")
end_date = dt.datetime.strptime("2019-12-31", "%Y-%m-%d")
df = client.get_time_series(polygon, start_date, end_date, collection=SatelliteImageryCollection.MODIS, indicators=["NDVI"])
df.head()

In [ ]:
# Create structure before push values in Analytics Fabrics
values = []
for i in range(0,len(df)):
    prop = {
        "Timestamp": str(df["value"].index[i]),
        "Values": {
            "NDVI": df["value"].values[i]
        }
        }
    values.append(prop)
values[0:5]

In [ ]:
# Push metrics in Analytics Fabrics
schema_id = "GeosysPy_NDVI"
client.push_metrics(polygon, schema_id, values)

In [ ]:
# Get metrics in Analytics Fabrics
start_date = dt.datetime.strptime("2018-01-01", "%Y-%m-%d")
end_date = dt.datetime.strptime("2022-04-01", "%Y-%m-%d")
client.get_metrics(polygon, schema_id, start_date, end_date)

In [2]:
# currently data retrived from mr_time_series are available only in PREPROD  
client = Geosys(API_CLIENT_ID, API_CLIENT_SECRET, API_USERNAME, API_PASSWORD, Env.PREPROD, Region.NA)

str_start_date="2020-10-09"
str_end_date="2022-10-09"
list_sensors=["Sentinel_2", "Landsat_8"]
bool_denoiser=True
str_smoother="ww"
bool_eoc=True
str_func="mean"
str_index="ndvi"
bool_raw_data=True
start_date="2020-10-09"
end_date="2022-10-09"
sensors=["Sentinel_2", "Landsat_8"]
denoiser=True
smoother="ww"
eoc=True
aggregation="mean"
index="ndvi"
raw_data=True
str_polygon="POLYGON ((-0.49881816 46.27330504, -0.49231649 46.27320122, -0.49611449 46.26983426, -0.49821735 46.27094671, -0.49881816 46.27330504))"

str_s3_path = client.get_mr_time_series(str_polygon, str_start_date= start_date, 
                                        str_end_date=end_date, list_sensors=sensors, bool_denoiser=denoiser, 
                                        str_smoother=smoother, bool_eoc=eoc, str_aggregation=aggregation, 
                                        str_index=index, bool_raw_data=raw_data)
str_s3_path

's3://geosys-geosys-us/2tKecZgMyEP6EkddLxa1gV/mrts/0c3f552bbb1f4257994aeb5349988048'

In [ ]:
str_polygon="POLYGON ((-0.49881816 46.27330504, -0.49231649 46.27320122, -0.49611449 46.26983426, -0.49821735 46.27094671, -0.49881816 46.27330504))"

str_s3_path = client.get_mr_time_series(str_polygon)
str_s3_path

In [ ]:
import boto3
import os
from urllib.parse import urlparse


def download_s3_files(str_s3_path):
    s3 = boto3.resource('s3')
    parsed_url = urlparse(str_s3_path)
    bucket_name = parsed_url.netloc
    directory_name = parsed_url.path.lstrip('/')
    bucket = s3.Bucket(bucket_name)
    for obj in bucket.objects.filter(Prefix=directory_name):
        if not os.path.exists(os.path.dirname(obj.key)):
            os.makedirs(os.path.dirname(obj.key))
        bucket.download_file(obj.key, obj.key)

download_s3_files(str_s3_path)